<h1> DattoRMM - Patch Policy - Data Ingestion - MongoDB</h1>

## Import Modules

In [ ]:
import datetime
import re
import os
import pandas as pd
import csv

## Arguments and Declarations


In [ ]:
# Selected Columns from Master Device View Export CSV - This is the column mask that will be used to trimp the outer merge on match key column
std_columns = [
    'Device UID',
    'Site Name',
    'Site UID',
    'Device Hostname',
    'Create Date',
    'Last Seen',
    'Last Audit Date',
    'Policy',
    'Schedule',
    'Last Run',
    'Online Duration (hrs)',
    'Last Reboot',
    'Group',
    'Source Modified Date',
    'Source Creation Date',
    'Source Filename'
 ]

In [ ]:
# review column filter for any duplicates values or close similarities
std_col_ser = pd.Series(std_columns).value_counts()
std_col_ser[std_col_ser > 1]

In [ ]:
# filename prefix timestamp format
time_format = '%Y_%m_%d_%H%M%S'

# define key column to join on
fieldnames_to_compare = 'Device UID'

# identify folder stages so that files are not called twice in the same stage
source_dir = 'd:/data_sets/raw/'

# Parse Date Data Options
date_parser = lambda c: pd.to_datetime(c, errors='coerce')
parse_dates =  ['Create Date', 'Last Seen']

# NA Values Check
na_values = ['Currently Online','null', '(null)']

# output of csv with matching key column
included_files = {}

# output csv of all files that could not be merged
excluded_files = {}

# Regex Match to group files to be combined on rows rather than merged on columns to prevent dropping rows if there isnt a key column match when files are combined in random order
pattern = re.compile(r'^(\w+)_')

# final dataframe before training and visualization
df_clean = pd.DataFrame(columns=std_columns)

# CSV File Types
devices_tab_export_filename = 'DeviceDetailsExport'
devices_tab_export_files = []
df_devices_list = []




manage_tab_export_filename = 'SystemDeviceSelection'
manage_tab_export_files = []
df_manage_list = []

<h1>Read all files in source_dir and sub directories</h1>
    <h3> Filter by '.csv' </h3>

In [ ]:
# pull all filenames walking through all folders (recursive going down the tree)
#all_source_csv = []
source_csv_dict = {}
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if '.csv' in file:
            #all_source_csv.append(os.path.join(root,file))
            source_csv_dict.update({os.path.join(file):os.path.join(root,file)})

# print(all_source_csv)
print('All CSV Files found before futher vetting and filtering')
print('='*50)
for file in source_csv_dict:
    print(file)

<h1>Sorting and Excluding Files</h1>

## Read all csv file columns and create two lists of files:
### Those with the chosen merge key column will be kept and the remaining filenames will not be called any further

In [ ]:
# for each filename - pull source data using source_info module in custom modules folder
#for filename in all_source_csv:

    # read in dataframe
    #df = pd.read_csv(filename)
    #print(df['Site Name'].unique())

    # for each df review for join key column to be present and add to included_files else add to excluded_files
    #print(df.columns)

   # compare_keys(df,filename)
for k,v in source_csv_dict.items():
    df = pd.read_csv(v)
    #print(df['Site Name'].unique())
    #compare_keys(df,filename)
    if fieldnames_to_compare not in df.columns:
        print(f'Missing Key: {fieldnames_to_compare} to Join in {filename}')
        excluded_files.update({k:v})
    else:
        included_files.update({k:v})

In [ ]:
print('Files with CORRECT join key column:')
print('-'*50)
for file in included_files:
    print(file)
print('='*50)

print('Files MISSING join key column:')
print('-'*50)
for file in excluded_files:
    print(file)
print('='*50)

## Parse Accepted CSV's for file discription and store as dictionary key pair

In [ ]:
pattern = re.compile(r'^([a-zA-Z]{0,})(\_|\-|''){0,1}([a-zA-Z]{0,})')

In [ ]:
for k,v in included_files.items():
    matches = pattern.search(k)
    if matches[1] == devices_tab_export_filename:
        print(f'''['{v}'] matches: ['{devices_tab_export_filename}'] on ['{matches[1]}']''')
        devices_tab_export_files.append({'filename':v,'groupname':manage_tab_export_filename})
    elif matches[3] == manage_tab_export_filename:
        print(f'''['{v}'] matches: ['{manage_tab_export_filename}'] on ['{matches[3]}']''')
        manage_tab_export_files.append({'filename':v,'groupname':manage_tab_export_filename})


print('='*50)

## For those files that have the key column, set index col and add source info
### 1. Add source file data as columns at end of dataframe (record the file creation, modified, and fullpath name)
### 2. Set index col = fieldnames_to_compare variable list

In [ ]:
def map_source(source_file):
    # pull source time from file properties
    source_info = srctime(source_file)

    # Import CSV
    df = pd.read_csv(source_file,index_col=fieldnames_to_compare)

    # add source info to new columns k with values v
    for k,v in source_info.items():
        #print('='*50)
        #print(f'key = {k}')
        #print(f'value = {v}')
        #print('='*50)
        df[k] = v
        #print(df['Source Creation Date'])
    return df

## Attempt 2 at updating data correctly on import.  Attempting to use iloc for each df row

In [ ]:
# Create blank DataFrame to fill
df_master = pd.DataFrame(columns=std_columns)

In [ ]:
test_details = pd.read_csv('.csv')

In [ ]:
test_manage = pd.read_csv('.csv')

In [ ]:
for row in df_test1.row:
    print(row)

In [ ]:
for file in devices_tab_export_files:
    #print(file['filename'])
    #print(file['groupname'])
    df_devices_list.append(map_source(file['filename']))


## Attempt 1 at updating data correctly on import.  Still having issues with micromanaging file order on join

In [ ]:
#print(manage_tab_export_files)
#print(devices_tab_export_files)
for file in devices_tab_export_files:
    #print(file['filename'])
    #print(file['groupname'])
    df_devices_list.append(map_source(file['filename']))




for file in manage_tab_export_files:
    #print(file['filename'])
    #print(file['groupname'])
    df_manage_list.append(map_source(file['filename']))


#print(df_manage_list)

In [ ]:
df_devices = pd.concat(df_devices_list,axis=0)
df_manage = pd.concat(df_manage_list,axis=0)

In [ ]:
df_manage.info()

In [ ]:
df_manage

In [ ]:
df_devices.info()

<h1>Join, Concat, and Merge</h1>

## Those of the same name should be concatonated by row or stacked

## Merge all dataframes (csv's) into an empty dataframe that contains all columns without data

In [ ]:
# as a df (dataframe) must be merged on another, we start with filelist with the first element as the df all will be merged into index[0] in list dtype
df_clean = df_list[0]

for df_object in df_list[1:]:
    # join on key column or columns (original 'set' dtype must be changed to 'list' to fit pandas expected argument for 'merge' method)
    df_clean.merge(df_object, on=(fieldnames_to_compare), how= 'outer',suffixes=('', '_drop'))

In [ ]:
# drop duplicate colummns renamed as '_drop' during parse
df_clean.drop([col for col in df_clean.columns if 'drop' in col], axis=1, inplace=True)

## Trim any columns not in the column standars list 'columns'

In [ ]:
len(df_clean.columns)

In [ ]:
df_clean.drop([col for col in df_clean.columns if col not in std_columns], axis=1, inplace=True)

In [ ]:
len(df_clean.columns)

In [ ]:
# review column filter for any duplicates columns and drop one
clean_col = dict(df_clean.columns.value_counts())
dup_cols = list({k for (k,v) in clean_col.items() if v > 1})
if dup_cols:
    df_clean.rename(dup_cols,axis=1,inplace=True)
df_clean

## Using lists 'parse_dates' as datetime column targets and 'data_parser' as the datetime function to be applied to each value in target columns along each row that will change value type

In [ ]:
# convert dates to datetime
# variable 'time_format' stated at declaration
df_clean[parse_dates] = df_clean[parse_dates].apply(date_parser)

## Add 'Offline 30 days' and 'Extended Reboot' Columns as datetime delta calculations from day this report is run

In [ ]:
# Fill in NaT / NaN data with 0 time so columns can be converted to datetime and datetime methods can be used
df_clean[parse_dates].fillna(pd.Timedelta('0 days'),inplace=True)

# Filter - Devices Offline 30 Days
df_clean['Offline 30 Days'] = df_clean['Last Seen'] > datetime.datetime.now() - pd.to_timedelta("30day")

# Filters - Last Reboot Extended Duration and Online without Reboot Extended Duration
df_clean['Last Reboot Extended'] = df_clean['Last Reboot'] > datetime.datetime.now() - pd.to_timedelta("30day")

## Apply heatmap to review any NaN or NaT (null) values before they can be dropped

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(data = df_clean.isnull(),yticklabels=False,cbar=False,cmap='plasma')

<h1>Start ML Trials</h1>

In [ ]:
## Create MS Patching pairplot DataFrame
df_patch_pair = df_clean

## Convert 'Category' columns into numbers to get value relationships

In [ ]:
d_patch_status = pd.get_dummies(df_clean['Patch Status'],prefix='d',prefix_sep='_')
df_patch_pair.drop('Patch Status',axis=1,inplace=True)
df_patch_pair = pd.concat([df_patch_pair,d_patch_status])

In [ ]:
d_offline_30days = pd.get_dummies(df_clean['Offline 30 Days'],drop_first=True,prefix='d',prefix_sep='_')
d_offline_30days.rename(columns={'d_True':'d_Offline_30 Days'},inplace=True)
df_patch_pair.drop('Offline 30 Days',axis=1,inplace=True)
df_patch_pair = pd.concat([df_patch_pair,d_offline_30days])

In [ ]:
d_last_reboot_ext = pd.get_dummies(df_clean['Last Reboot Extended'],drop_first=True,prefix='d',prefix_sep='_')
d_last_reboot_ext.rename(columns={'d_True':'d_Last Reboot Extended'},inplace=True)
df_patch_pair.drop('Last Reboot Extended',axis=1,inplace=True)
df_patch_pair = pd.concat([df_patch_pair,d_last_reboot_ext])

In [ ]:
d_av_status = pd.get_dummies(df_clean['Antivirus Status'],drop_first=True,prefix='d',prefix_sep='_')
d_av_status.rename(columns={'d_Running & up-to-date':'d_AV Status Ok'},inplace=True)
df_patch_pair.drop('Antivirus Status',axis=1,inplace=True)
df_patch_pair = pd.concat([df_patch_pair,d_av_status])

In [ ]:
d_reboot_required = pd.get_dummies(df_clean['Reboot required'],drop_first=True,prefix='d',prefix_sep='_')
d_reboot_required.rename(columns={'d_True':'d_Reboot required'},inplace=True)
df_patch_pair.drop('Reboot required',axis=1,inplace=True)
df_patch_pair = pd.concat([df_patch_pair,d_reboot_required])
d_reboot_required

In [ ]:
sns.pairplot(data=df_patch_pair)

In [ ]:
for col in df_patch_pair.columns:
    print(col)

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(30,15))
sns.lineplot(data=df_patch_pair,x='Last Reboot',y='Patches Approved Pending',lw=.5)
plt.savefig('.png')

In [ ]:
df_patch_pair['Online Duration (hrs)'].value_counts()

In [ ]:
df_patch_pair['Site Name'].unique()

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(30,15))
sns.barplot(data=df_patch_pair,x='Site Name',y='Online Duration (hrs)')

In [ ]:
# store filename info as dictionary
var_dict = {}
i = 0
df_list = []
df_names =[]


# create variable dictionary before import iteration
for filename in all_shaped_csv:
    var_dict.update({('df' + str(i)): (shaped_dir + filename)})
    i = i + 1

# for filename = key(k) import into pandas and append resulting dataframe to df list as element
for k, v in var_dict.items():
    k = pd.read_csv(v)
    df_names.append(v)
    df_list.append(k)

#print(df_names)

# as a df (dataframe) must be merged on another, we start with filelist with the first element as the df all will be merged into index[0] in list dtype
for df_object in df_list[1:]:
    #print(df_object)
    #print("")

    # join on key column or columns (original 'set' dtype must be changed to 'list' to fit pandas expected argument for 'merge' method)
    df_list[0].merge(df_object, on=list(fieldnames_to_compare), how= 'outer')
    print(df_list[0]['Policy'])


# add current timestamp to filename for reference
current_time = (datetime.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# add 'merged_' to filename startswith and export
df_list[0].to_csv(merged_dir + 'merged_' + str(current_time) + ".csv", index= False)

cleanup = True
# clean up intermediate data
if cleanup == True:
    for s in all_shaped_csv:
        path = shaped_dir + s
        print(f'Removing file {path}')
        os.remove(path)







In [ ]:
df

In [ ]:
for filename in all_source_csv:
    source_info = srctime(source_dir + filename)
    df = pd.read_csv(source_dir + filename)
    for k,v in source_info.items():
        print(f'The key is {k}')
        print(f'The value is {v}')
        df[k] = v
    #df.insert(1,columns=source_info.keys(),source_info.values())
    #df[source_info.keys()]
    #print(source_info)
    #print(source_info.keys())
    #print(source_info.values())


    #print(filename)

    #df.colums = fs.patch_columns
    #print(df.columns)

In [ ]:
df

# Import Modules

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import json


# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

# import configparser for env secrets
from configparser import ConfigParser
config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# DattoRMM - Pull JSON from API

## Create Auth Token

### Create DataFrame via API Call Iteration


In [ ]:
# DattoRMM DataFrame - import and assign secrets from env.ini
base_uri = config['dattormm']['base_uri']
api_key = config['dattormm']['api_key']
api_secret = config['dattormm']['api_secret']
# call token api url
token_uri = config['dattormm']['token_uri']

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = api_key
data['password'] = api_secret

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']
## Create Devices DataFrame
# request content response
request_url = f'{base_uri}/account/devices'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)
print(c_dict['pageDetails']['nextPageUrl'])

# iterate and combine remaining pages
df_devices = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)
    print(c_dict['pageDetails']['nextPageUrl'])
    df_current_page = pd.DataFrame(c_dict['devices'])
    df_devices = pd.concat([df_devices, df_current_page], ignore_index=False)

# Data Shaping

## Create New Columns from Dictionary Columns

### Set Index to device UID

In [ ]:
df_devices.set_index('uid',inplace=True)

### Type | Category

In [ ]:
def device_category(device):
    if device is None:
        return None
    else:
        return device['category']

In [ ]:
def device_type(device):
    if device is None:
        return None
    else:
        return device['type']

In [ ]:
df_devices['category'] = df_devices['deviceType'].apply(device_category)
df_devices['type'] = df_devices['deviceType'].apply(device_type)

# Rename 'type' values to split devices into (2) : 'computer' or 'server'
#df_devices['type'].replace({'Desktop':'computer','Laptop':'computer','Server':'server'},inplace=True)

In [ ]:
df_devices.drop(columns='deviceType',inplace=True)

### Patch Management Breakdown
 patchStatus | patchesApprovedPending | patchesNotApproved | patchesInstalled

In [ ]:
# patchStatus
def patch_status(patch_management):
    return patch_management['patchStatus']

df_devices['patchStatus'] = df_devices['patchManagement'].apply(patch_status)

# patchesApprovedPending
def patches_approved_pending(patch_management):
    return patch_management['patchesApprovedPending']

df_devices['patchesApprovedPending'] = df_devices['patchManagement'].apply(patches_approved_pending)

# patchesNotApproved
def patches_not_approved(patch_management):
    return patch_management['patchesNotApproved']

df_devices['patchesNotApproved'] = df_devices['patchManagement'].apply(patches_not_approved)

# patchesInstalled
def patches_installed(patch_management):
    return patch_management['patchesInstalled']

df_devices['patchesInstalled'] = df_devices['patchManagement'].apply(patches_installed)


# drop patchManagement {inplace=True}
df_devices.drop('patchManagement',axis=1,inplace=True)

## Drop 'antivirus'

In [ ]:
df_devices.drop('antivirus',axis=1,inplace=True)

## Create Time Columns and Timedate Shaping

### Add Timezone Column from UDF

In [ ]:
# Timezone
def local_timezone(udf):
    return udf['udf10']

df_devices['localTimezone'] = df_devices['udf'].apply(local_timezone)

# drop udf {inplace=True}
df_devices.drop('udf',axis=1,inplace=True)

### Create Date Correlation Columns

In [ ]:
# all date columns
parse_dates =  ['lastAuditDate','lastSeen','lastReboot','creationDate',]

### Convert Epoch to UTC

In [ ]:
df_devices['lastAuditDate'] = pd.to_datetime(df_devices['lastAuditDate'],unit='ms',errors='coerce')
#df_devices['lastAuditDate'].head(5)

In [ ]:
df_devices['lastSeen'] = pd.to_datetime(df_devices['lastSeen'],unit='ms',errors='coerce')
#df_devices['lastSeen'].head(5)

In [ ]:
df_devices['creationDate'] = pd.to_datetime(df_devices['creationDate'],unit='ms',errors='coerce')
#df_devices['creationDate'].head(5)

In [ ]:
df_devices['lastReboot'] = pd.to_datetime(df_devices['lastReboot'],unit='ms',errors='coerce')
#df_devices['lastReboot'].head(5)

### Define and apply functions to create correlation columns

In [ ]:
def no_audit_7_days(last_audit):
    if last_audit < dt.datetime.now() - dt.timedelta(days=7):
        return 1
    else:
        return 0

In [ ]:
def offline_30_days(last_seen):
    if last_seen < dt.datetime.now() - dt.timedelta(days=60):
        return 1
    else:
        return 0

In [ ]:
def no_reboot_60_days(last_reboot):
    if last_reboot < dt.datetime.now() - dt.timedelta(days=60):
        return 1
    else:
        return 0

In [ ]:
# Create Column - Devices Last Audit > 7 days
df_devices['noAudit7Days'] = df_devices['lastAuditDate'].apply(no_audit_7_days)

In [ ]:
# Create Column - Devices Offline 30 Days
df_devices['offline30Days'] = df_devices['lastSeen'].apply(offline_30_days)

In [ ]:
# Create Column - Last Reboot Extended Duration and Online without Reboot Extended Duration
df_devices['noReboot60Days'] = df_devices['lastReboot'].apply(no_reboot_60_days)

In [ ]:
# Create DF copy for reference
df_raw_data = df_devices

## DattoRMM DataFrame Data Standardization Shaping

### Hostname to_upper()

In [ ]:
df_devices['hostname'] = df_devices['hostname'].str.upper()

### Replace Values {'TRUE':1,'FALSE':0}

In [ ]:
df_devices.replace({True:1,False:0},inplace=True)

In [ ]:
convert_to_int = df_devices.dtypes[(df_devices.dtypes == 'float') | (df_devices.dtypes == 'bool') | (df_devices.dtypes == 'uint8')].index.tolist()

In [ ]:
df_devices[convert_to_int] = df_devices[convert_to_int].astype('Int64')

### Drop Columns with no Data (NaN, 0, or None as only Value on all rows)

In [ ]:
df_devices.drop(['lastLoggedInUser','warrantyDate'],axis=1,inplace=True)

## Add 'patchStatus' Dummy Columns

In [ ]:
df_patch_status = pd.get_dummies(df_devices['patchStatus'],prefix='patchStatus')
df_patch_status.drop('patchStatus_NoPolicy',axis=1, inplace=True)
df_devices = df_devices.join(df_patch_status)
df_devices.drop('patchStatus',axis=1,inplace=True)

# MongoDB

In [ ]:
username = config['mongodb']['username']
password = config['mongodb']['password']
connection_ip = config['mongodb']['connection_ip']
database = 'seed_data'

# import bson for object encoding
import bson

# Provide the mongodb atlas url to connect python to mongodb using pymongo
#CONNECTION_STRING = f"mongodb://{username}:{password}@{connection_ip}/{database}"
CONNECTION_STRING = 'mongodb://localhost:27017'


# Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
from pymongo import MongoClient
client = MongoClient(CONNECTION_STRING)


db = client['datto_rmm']
collection = db['seed_data']


In [ ]:
df_ingest = df_devices.reset_index()

In [ ]:
dt_list = list(df_ingest.dtypes[df_ingest.dtypes == 'datetime64[ns]'].index)

In [ ]:
df_ingest.fillna(pd.NA,inplace=True)

In [ ]:
entries = []
cols = enumerate(list(df_ingest.columns))

for index, row in df_ingest.iterrows():
    current_row = row.to_dict()
    new_entry = {}
    try:
        try:
            new_entry['onlineData'] = {'legend':['onlineStatus','lastSeen'],'data':[current_row['online'],current_row['lastSeen']]}
        except:
            print(f'Unable to create column: "onlineData"')
        try:
            if current_row['suspended'] == 1:
                new_entry['isSuspended'] = {'legend':['suspendedStatus','lastSeen'],'data':[current_row['suspended'],current_row['lastSeen']]}
        except:
            print(f'Unable to create column: "isSuspended"')
            print(current_row['suspended'])
        try:
            if current_row['deleted'] == 1:
                new_entry['isDeleted'] = {'legend':['deletedStatus','lastSeen'],'data':[current_row['deleted'],current_row['lastSeen']]}
        except:
            print(f'Unable to create column: "isDeleted"')
            print(current_row['deleted'])
        try:
            if current_row['rebootRequired'] == 1:
                new_entry['rebootRequired'] = {'legend':['rebootRequired','lastReboot'],'data':[current_row['rebootRequired'],current_row['lastReboot']]}
        except:
            print(f'Unable to create column: "rebootRequired"')
            print(current_row['rebootRequired'])
        for k,v in current_row.items():
            if pd.isna(v) == True:
                continue
            elif (k == 'online') | (k == 'lastSeen') | (k == 'suspended') | (k == 'rebootRequired') | (k == 'lastReboot') |(k == 'deleted'):
                continue
            else:
                new_entry[k] = v

        entries.append(new_entry)
    except:
        print(current_row)

In [ ]:
entries[55]['hostname']

In [ ]:
collection.insert_one(entries[1])

In [ ]:
result = collection.find_one({'uid':entries[0]['uid']})
result['_id']
match_id = result['_id']
k = 'onlineStatus'
v = {"legend":'test'}
collection.update_one({'_id':match_id},{'$set': {f"{ str(k)  + '.legend'}" : v['legend'] }},upsert=False)

In [ ]:


for entry in entries:
    try:
        result = collection.find_one({'uid':entry['uid']})
        print(f"_id found: {result['_id']}")
        match_id = result['_id']
        for k,v in entry.items():
            try:
                #print(k,v)
                if k in  ['onlineData','isSuspended','isDeleted','rebootRequired']:
                    try:
                        #print({f"{ str(k)  + '.legend'}": v['legend']})
                        #collection.insert_one({'_id':m_id}, {f"{str(k)}":v['legend']})
                        collection.update_one({'_id':match_id},{'$set': {f"{ str(k)  + '.legend'}" : v['legend'] }},upsert=False)
                        collection.update_one({'_id':match_id},{'$push': {f"{ str(k)  + '.data'}" : {"$each":  v['data'] }}},upsert=True)
                        #print(f"Successfully updated {k} for {entry['hostname']}")
                    except Exception as error:
                        print(error)
                else:
                    try:
                        collection.update_one({'_id':match_id}, {'$set':{f"{str(k)}":v}},upsert=True)
                        #print(f"Successfully inserted new {k} for {entries[0]['hostname']}")
                    except Exception as error:
                        #print(f"Unable to insert new {k} for {entries[0]['hostname']}")
                        print(error)
            except Exception as e:
                print(e)
                continue
    except Exception as e:
        try:
            print(f"_id not found: attempting new entry for {entry['hostname']} with uid: {entry['uid']}")
            collection.insert_one(entry)
        except:
            print(e)

print('*'*120)
print('End of Import!')